In [1]:
import numpy as np
import os
import open3d as o3d
import cloudComPy as cc


from yaml import load
from yaml.loader import Loader

from helpers.utils import load_matrix_from_file, suggest_global_shift
from helpers import pairwise_registration as pr

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
PROCESSING_FILEPATH = "../../real_data/09_GrotteDeVallorbe/params.yaml"

In [4]:
p = load(open(PROCESSING_FILEPATH), Loader)

## Read the datset into memory and optionally display it with Open3D

In [5]:
# define the file path, root should point to the home cave directory. 
root = p["paths"]["root"]
RAW_CLOUD_FILEPATH = os.path.join(root, p["paths"]["inCloudName"])

print(RAW_CLOUD_FILEPATH)
# load the point cloud to memory 
cloud = cc.loadPointCloud(RAW_CLOUD_FILEPATH)

F:/ScanLeica/from_pointcloud_to_mesh/real_data/09_GrotteDeVallorbe\raw/GrotteDeVallorbe.las


In [6]:
# print the cloud size 
cS = cloud.size()

print(cS)

562385971


In [6]:
# load the point cloud as an open3d geometry.
pcd = o3d.io.read_point_cloud(RAW_CLOUD_FILEPATH)

viz = o3d.visualization.draw_geometries([pcd])

## Calculate the transformation matrix for the registration of the cloud

In [14]:
# load the targets in the lidar scan. 
SCAN_TARGETS_FILEPATH = os.path.join(root, p["paths"]["scanTargets"])
SKIP_ROWS = p["alignment"]["skip_rows"]
MAX_ROWS = p["alignment"]["max_rows"]
USECOLS = p["alignment"]["usecols"]
scan_targets = np.loadtxt(SCAN_TARGETS_FILEPATH, 
                          skiprows=SKIP_ROWS, 
                          max_rows=MAX_ROWS,
                          usecols=USECOLS, 
                          delimiter=",")

In [15]:
# load the targets from the therion survey.
THERION_TARGETS_FILEPATH = os.path.join(root, p["paths"]["therionTargets"])

therion_targets = np.loadtxt(THERION_TARGETS_FILEPATH,
                             skiprows=SKIP_ROWS, 
                             max_rows=MAX_ROWS,
                             usecols=USECOLS, 
                             delimiter=",")

In [16]:
globalShift = suggest_global_shift(therion_targets)
print(globalShift)

therion_targets_shifted = therion_targets + globalShift.T

[[-516300.]
 [-172100.]
 [   -800.]]


In [17]:
therion_targets_shifted

array([[ 90.66, -14.14,  27.68],
       [ 76.99, -15.74,  27.49],
       [ 54.37, -28.32,  22.74],
       [ 33.16, -31.51,  18.14],
       [  6.59, -31.23,  11.93],
       [ -6.22, -43.08,   9.85],
       [-21.44, -54.  ,  11.85],
       [-46.05, -65.57,  12.57]])

In [18]:
result, R, T = pr.pairWiseRegistration(scan_targets,therion_targets_shifted, atol= 3e-1)

transformation is correct!


In [19]:
# RMS error ? 
RMSE = np.sqrt( np.sum( (result.T - therion_targets_shifted)**2 ) / therion_targets_shifted.shape[0] )
print(RMSE)

0.11715910103273558


In [20]:
# write the matrix string.

transformationMatrix = np.diag((1.,1.,1.,1.))
transformationMatrix[:3,3] = T.T
transformationMatrix[0:3,0:3] = R

In [21]:
# write the global shift matrix string.

globalShiftMatrix = np.diag((1.,1.,1.,1.))
globalShiftMatrix[:3,3] = globalShift.T

In [22]:
# save to predefined paths 
TR_MATRIX_FILEPATH = os.path.join(root, p["paths"]["transformMatrix"])
GLOBAL_SHIFT_FILEPATH = os.path.join(root, p["paths"]["globalShift"])
np.savetxt(TR_MATRIX_FILEPATH, transformationMatrix)
np.savetxt(GLOBAL_SHIFT_FILEPATH, globalShiftMatrix)

## Apply the transformation matrix to the point cloud

In [23]:
# read the transformation as a CloudCompare transformation matrix. 
tr_matrix_as_string = load_matrix_from_file(TR_MATRIX_FILEPATH)
trans = cc.ccGLMatrix.fromString(tr_matrix_as_string)

In [24]:
# apply the rigid transformation 
cloud.applyRigidTransformation(trans)

In [25]:
# resample the cloud spatially so as to perform quicker computation
SPATIAL_SAMPLING_DISTANCE = p["subsampling"]["spatialSamplingDistance"]

refCloud = cc.CloudSamplingTools.resampleCloudSpatially(cloud, SPATIAL_SAMPLING_DISTANCE)

(spatiallyResampledCloud, res) = cloud.partialClone(refCloud)
spatiallyResampledCloud.setName("spatiallyResampledCloud")


In [26]:
# apply the global shift translation
x,y,z = globalShift
print(x, y, z)
cloud.setGlobalShift(x,y,z)
# apply the rigid transformation 
spatiallyResampledCloud.setGlobalShift(x,y,z)

[-516300.] [-172100.] [-800.]


## Save the spatially subsampled point cloud

In [28]:
SUBSAMPLED_CLOUD_FILEPATH = os.path.join(root, p["paths"]["subsampledGeorefOutCloudName"])

ret = cc.SavePointCloud(spatiallyResampledCloud, SUBSAMPLED_CLOUD_FILEPATH)


## Save the whole point cloud

In [30]:
GEOREF_CLOUD_FILEPATH = os.path.join(root, p["paths"]["georefOutCloudName"])

ret = cc.SavePointCloud(cloud, GEOREF_CLOUD_FILEPATH)


## Crop the entrance area using a predefined line 

In [46]:
CROP_POLYLINE_PATH = os.path.join(root, p["paths"]["entranceCrop"])
CROP_DIRECTION = p["entranceCropping"]["direction"]
LEAVE_INSIDE =  p["entranceCropping"]["leaveInside"]

print(CROP_POLYLINE_PATH)
cropPolyLine = cc.loadPolyline(CROP_POLYLINE_PATH, cc.CC_SHIFT_MODE.XYZ, 0,-x,-y,-z)

cropPolyLine.setClosed(True)
CloudCropZ = cloud.crop2D(cropPolyLine, CROP_DIRECTION, LEAVE_INSIDE) # 2 means cropping in the Z direction. 
                                  

../../real_data/06_GrandFontannet/process/CropEntrance.poly


In [47]:
spatiallyResampledCloudCropZ = spatiallyResampledCloud.crop2D(cropPolyLine, CROP_DIRECTION, LEAVE_INSIDE) # 2 means cropping in the Z direction. 

## Save the files

In [48]:
# define the output filepath
CUT2D_CLOUD_FILEPATH = os.path.join(root, p["paths"]["Cut2DOutCloudName"])

# save the point cloud
ret = cc.SavePointCloud(CloudCropZ, CUT2D_CLOUD_FILEPATH)

In [49]:
# define the output filepath 
SUBSAMPLED_CUT2D_CLOUD_FILEPATH = os.path.join(root, p["paths"]["subsampledCut2DOutCloudName"])

# save the point cloud
ret = cc.SavePointCloud(spatiallyResampledCloudCropZ, SUBSAMPLED_CUT2D_CLOUD_FILEPATH)